# Se conectando com o Google Drive


In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
%cd '/gdrive/My Drive/PDFs_ANA'
%ls

/gdrive/My Drive/PDFs_ANA
ANAC.csv                    Resolucao_ANAC_parte4.txt
portarias_antigo.xlsx       Resolucao_ANAC_parte5.txt
Resolucao_ANAC_parte10.txt  Resolucao_ANAC_parte6.txt
Resolucao_ANAC_parte11.txt  Resolucao_ANAC_parte7.txt
Resolucao_ANAC_parte1.txt   Resolucao_ANAC_parte8.txt
Resolucao_ANAC_parte2.txt   Resolucao_ANAC_parte9.txt
Resolucao_ANAC_parte3.txt


# Entrando com os pacotes necessários

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 904 kB 4.2 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/u

In [ ]:
!pip install pdf2image
!apt-get install -y poppler-utils
from pdf2image import convert_from_path
from PIL import Image

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 90 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 1s (104 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 148860 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
pip install easyocr

     |████████████████████████████████| 63.6 MB 19 kB/s 
     |████████████████████████████████| 37.1 MB 54 kB/s 


# Entrando com a base de dados

In [ ]:
import pandas as pd

df = pd.read_excel('ANAC.csv', sheet_name='Portarias +Decisões + IACs')

# Separando quais portarias estão revogadas

In [ ]:
revogada = list(df['Revogada'])[0:3]

lista_revogada = []

for i in range(0,len(revogada)):
    if 'Revogada' in str(revogada[i]):
        lista_revogada.append('True')
    else:
        lista_revogada.append('False')

# Arrumando os links presentes no excel

In [ ]:
lista_link = []
for i in range(0,len(df['Hiperlink_Excel'])):
  lista_link.append(df['Hiperlink_Excel'][i].replace(' ',''))


# Realizando os downloads dos PDF's

In [ ]:
from selenium import webdriver
from pdf2image import convert_from_path
from IPython.display import Image

import pandas as pd
import easyocr
import os

import time

# initialise browser
#browser = webdriver.Chrome(os.getcwd()+'/chromedriver')

# Entrando com o banco de dados

url_list = lista_link[0:3]
            
texto_final = []
Url_error = []

for i in range(0,len(url_list)):

  try:

    import sys
    sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
    from selenium import webdriver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

    browser.get(str(url_list[i]).replace('[','').replace(']','').replace("'",""))

    # Fazendo Download dos arquivos em PDF

    download = browser.find_element_by_xpath('//*[@id="icon"]/iron-icon')

    download.click()

  except:
    print(url_list[i])
    Url_error.append(url_list[i])

  time.sleep(20)

  # Transformando as páginas em jpg

  # Pegando uma lista de todos os documentos presentes na pasta

  arquivos = !ls '/gdrive/My Drive/PDFs_ANA'

  # Arrumando a lista 

  lista_arquivo = []
  for i in range(0,len(arquivos)):
    result = arquivos[i].replace('[','').replace(']','').replace("'","").replace(',','').replace('\t','').split('  ')
    lista_arquivo.append(result)
 
  lista_arquivo =[item for sublist in lista_arquivo for item in sublist]

  pdf_list = [x for x in lista_arquivo if x.endswith(('PDF'))]

  i = 0

  for pdf in pdf_list:

    pages = convert_from_path(pdf, 500)

    for page in pages:
      i = i+1
      page.save('out'+str(i)+'.jpg', 'JPEG')

  time.sleep(20)


  # Pegando uma lista de todos os documentos presentes na pasta

  arquivos = !ls '/gdrive/My Drive/PDFs_ANA'
  lista = str(arquivos).replace('\\t',' ').split(' ')

  # Arrumando a lista 

  lista_arquivo = []
  for i in range(0,len(lista)):
    result = lista[i].replace('[','').replace(']','').replace("'","").replace(',','')
    lista_arquivo.append(result)

  # Pegando apenas os arquivos de jpg

  imagens_lista = [x for x in lista_arquivo if x.endswith(('jpg', 'png'))]
  imagens_lista.sort()
  
  # Separando o conteudo textual presente nas imagens

  temporario = []
  texto = []
  for i in range(0,len(imagens_lista)):

    reader = easyocr.Reader(['pt'])
    resultado = reader.readtext(imagens_lista[i])

    # Formatando a parte textual 

    texto_parte = []
    for i in range(0,len(resultado)):
      result = str(resultado[i]).split("'")[1].split("'")[0]
      texto_parte.append(result)

      texto = []
      texto.append(" ".join(str(x) for x in texto_parte))

    print(texto)

    temporario.append([''.join(texto[:len(texto)])])

    flat_list = [item for sublist in temporario for item in sublist]

  texto_final.append(temporario)

  # Removendo o conteudo presente no Drive

  !rm ./*PDF
  !rm ./*jpg

  time.sleep(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: use options instead of chrome_options


https://pergamum.anac.gov.br/arquivos/PD1962-0190.PDF
['Quarta-feira 7 DIÂRIO  OFICIAL (Seção Parte I) Novembro de 1962 11585 955 4.956 = 4,992 4,,994 Retificação III Pesos Máxin%sde 4.995 5.300 5.305 ~ 5.306 Operação 6.307 5.308 ~ 5.309 ~ 5.310 EXPEDIENTE DO MINISTRO 5.311 5.313 ~ 5.314 ~ 5.315 Nia matéria em epígrafe, ` publicade 6.316 5.317 ~ 5.318 5.319 no Diário Oficial   de 22 de outubro 5.320 5.321 2 5.322 ~ 5.323 de 1962, às páginas números 11.088, Condlgões 5.324 " 6.325 5.326 5.327 na 4v coluna Aeronave Na decolagem No pouso 5 328 " 329 5 .330 5.,332 1) na 17v requerldas 6.333 5.334  5.335  5.336 Onde se lê: 5.,337 5.338 5 339 5.340 06 Jejanira Betista 5.341 5.342 5.343 5.275 Leia-se: 5 ,276 5.277 ~ 5.278  5,279 Dejalma", 0.9985499235341558) Batista 5.280 5.281 5.282 5.283 2) na 19ª linha, 5.284 5.285 5.286 5,287 Onde se lê: Quilos Quilos 5.288-62) Orlando do Couto Leia-se: Lockheed Eec - Em 19.10.1962 Orlando ` Azevedo do Couto_ tra 188-A 41.256 43 .386 Na Jorge da Silva Prz

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: use options instead of chrome_options


https://pergamum.anac.gov.br/arquivos/PD1972-0017.PDF
rm: cannot remove './*PDF': No such file or directory
rm: cannot remove './*jpg': No such file or directory


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: use options instead of chrome_options


https://pergamum.anac.gov.br/arquivos/PD1992-0333.PDF
['(VII/ ElIO", 0.4767417965353433) DA {  ; /", 0.10626833933181397) GABiNETE Do MINIsTRO SECRETARIA Publicado no Diário Oficlal de 213 SET 1992 SERVIÇO PÚBLICO FEDERAL MINISTÉRIO DA AERONÁUTICA ]OU 1 82 DEPARTAMENTO DE AVIAÇAO CIVIL POR TAR IA Ne 333/DGAC , de 21 de setembr0 de 1 992 Fixa índ i ces Para reajuste da5 tar i fa5 cas", 0.5100339851350045) da infra- estrutura aeronaut ica e dà outras Providênc ias _ D IRETOR-GERAL D0 DEPARTAMENTO DE AVIAçÃo CIVIL, de acordo Com 3 delegação de a", 0.42314007059698966) d0 MINISTRO Da AERONÁUTICA, outorgada P e ]a Port ar ; a n 9 236/GM-5 , de 26 de Mar € 0 de 1 990 , e , de conformi dade CoM 0 d i sPost 0 No Paragrafo úni co d0 art ig0 29 da Le i n9 6 . 009 , de 26 d € dezembr 0 de 1 973 , Paragrafo 1 9 do art /90 39 , d0 Decret0 n0 89 . 121 , de 06 de dezembr o de 1983 , da Portar i a n9 84/MEFP , de 1 8 de julho de 1 991 , e usando da5 atr ibu 1çães que 1he conferem 05 ig05 4 0 e 4 1 da 

# Estruturando o Dataframe

In [ ]:
import re 

# Número da Resolução

Num_portaria = []
for i in range(0,len(texto_final)):
    
    try:
    
        if 'Nª' in str(texto_final[i]):
            result = str(texto_final[i]).split('Nª')[1][:5]
            Num_portaria.append(result)

        elif 'nª' in str(texto_final[i]):
            result = str(texto_final[i]).split('nª')[1][:5]
            Num_portaria.append(result)

        elif 'N?' in str(texto_final[i]):

            result = str(texto_final[i]).split('N?')[1][:5]
            Num_portaria.append(result)

        elif 'N ?' in str(texto_final[i]):

            result = str(texto_final[i]).split('N ?')[1][:5]
            Num_portaria.append(result)

        elif 'NM' in str(texto_final[i]):

            result = str(texto_final[i]).split('NM')[1][:5]
            Num_portaria.append(result)

        elif 'N2' in str(texto_final[i]):

            result = str(texto_final[i]).split('N2')[1][:5]
            Num_portaria.append(result)

        elif 'NC' in str(texto_final[i]):

            result = str(texto_final[i]).split('NC')[1][:5]
            Num_portaria.append(result)

        elif 'PORTARIA No' in str(texto_final[i]):

            result = str(texto_final[i]).split('PORTARIA No')[1][:5]
            Num_portaria.append(result)


        elif 'Portaria H?' in str(texto_final[i]):

            result = str(texto_final[i]).split('Portaria H?')[1][:5]
            Num_portaria.append(result)

        else: 
            result = re.findall(r'\d+0', str(texto_final[i]))[0]
            Num_portaria.append(result)
            
    except:
        if re.findall(r'\d+', str(texto_final[i])) == []:
          Num_portaria.append(0000)
        else:
          result = re.findall(r'\d+', str(texto_final[i]))[0]
          Num_portaria.append(result)
        
Num_port = []
for i in range(0,len(Num_portaria)):
    try:
        result = re.findall(r'\d+', str(Num_portaria[i]))[0]
        Num_port.append(result)
        
    except:
        result = '00'
        Num_port.append(result)
        
# Ano da Resolução

Ano_portaria = []
for i in range(0,len(texto_final)):
    try:
        if '", 0.' in str(texto_final[i])[0:100]:
          result = re.findall(r'\d{4}', str(texto_final[i]).split(')')[1])[0]
          Ano_portaria.append(result)

        else:
          result = re.findall(r'\d{4}', str(texto_final[i]))[0]
          Ano_portaria.append(result)

    except:
        print("Ano não encontrado")
        Ano_portaria.append('2021')
    

Ano não encontrado


In [ ]:
# Criando o ID 

tipo = ['701']*len(texto_final) # Tipo de Lei

parte1 = [i + j for i, j in zip(tipo, Num_port)] # Juntado os valores
ID = [i + j for i, j in zip(parte1, Ano_portaria)] 

# Criando um DataFrame para alocar os outputs

BANCO = pd.DataFrame (ID ,columns=['ID'])
BANCO['Texto_lei'] = texto_final
BANCO['Data_lei'] = ['']*len(texto_final)
BANCO['Data_publicação'] = ['']*len(texto_final)
BANCO['Tipo_lei'] = tipo
BANCO['Revogada'] = lista_revogada
BANCO['Setor'] = ['ANA']*len(texto_final)

BANCO

,ID,Texto_lei,Data_lei,Data_publicação,Tipo_lei,Revogada,Setor
0,7011791962,[[Quarta-feira 7 DIÂRIO OFICIAL (Seção Parte ...,,,701,True,ANA
1,70102021,[],,,701,True,ANA
2,701101062,"[[(VII/ ElIO"", 0.4767417965353433) DA { ; /"",...",,,701,True,ANA


In [ ]:
# Exportando em formato TXT

BANCO.to_csv("Resolucao_ANAC_parte11.txt", index=False, encoding='utf-8-sig',sep = '字')